In [70]:
import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import Solver3 as sl 
from scipy.interpolate import interp1d

%load_ext autoreload

%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generating Delta Vector

In [71]:

def cov_matrix(sigma, num_points):
  sigma = (sigma ** 2) * np.eye(num_points)
  return sigma


def compute_basis_functions_at_x(x, mesh):
    """
    Given an observation point x and the mesh, compute the two nonzero
    finite element basis (hat) functions at x.
    
    Returns:
      k: the index such that x is in [mesh[k], mesh[k+1]]
      psi_left: the value of the basis function associated with mesh[k] at x.
      psi_right: the value of the basis function associated with mesh[k+1] at x.
    """
    mesh = np.asarray(mesh, dtype=float)
    # Find the index k such that x is in [mesh[k], mesh[k+1]]
    k = np.searchsorted(mesh, x) - 1
    k = np.clip(k, 0, len(mesh) - 2)  # ensure k is valid
    
    psi_left = sl.phi_i(k, x, mesh)
    psi_right = sl.phi_i(k+1, x, mesh)
    return k, psi_left, psi_right

# Example: Evaluate the FE solution at a set of observation points
def fe_solution_at_obs(c_sol, mesh, x_obs):
    """
    Compute the finite element solution at observation points x_obs,
    given the nodal solution c_sol and the mesh.
    
    Parameters:
      c_sol : array of nodal values (length N)
      mesh  : array of node coordinates (length N)
      x_obs : array of observation points
      
    Returns:
      c_interp: array of interpolated FE solution values at x_obs.
    """
    c_sol_full = sl.assemble_nodal_values(c_sol)
    c_sol_full_array = np.asarray(c_sol_full).flatten()
    c_interp = np.zeros_like(x_obs, dtype=float)
    
    for idx, x in enumerate(x_obs):
        k, psi_left, psi_right = compute_basis_functions_at_x(x, mesh)
        # The solution at x is the weighted average of the two nodal values
        c_interp[idx] = c_sol_full_array[k] * psi_left + c_sol_full_array[k+1] * psi_right
    return c_interp

# Alternatively, if many observation points fall in the same element,
# you could vectorize the process by grouping x_obs by element.



In [72]:
def add_noise(observations_at_xi, num_points, sigma):
    """
    Adds a normally distributed noise, theta
    to observations from the forward solver.

    Arguments:
    observations_at_xi : observations at predetermined xi using interplotion. 
    num_points : how big your covariance matrix is 

    Returns:
    Delta : Array of Noisy observations.
    
    """
    sigma = cov_matrix(sigma, num_points)
    noise = np.random.multivariate_normal(np.zeros(num_points), sigma)
    delta = observations_at_xi + noise 
    return delta



# Define Likelihood Function 

In [73]:
def phi(observations, predicted, sigma, num_points) :
    '''
    For a set of predetermined points xi -- obtained via np.linspace,
    this function defines the likelihood function 

    Arguments:
    observations: Generated noisy observation using beta_true -- corresponds to y in literature
    predicted: For a proposed beta_i, we compute the noisy observation using the forward solver 
    -- corresponds to g(beta_i) in literature

    Returns: 
    Likelihood function that is proportional to the prior distribution
    
    '''
    covariance_matrix = cov_matrix(sigma, num_points)
    diff = predicted - observations
    covariance_matrix_inv = np.linalg.inv(covariance_matrix) 
    val = 0.5 * diff.T @ covariance_matrix_inv @ diff
    return val



In [74]:
def compute_A(phi_0, phi_i, sigma):

    val = np.exp(phi_0 - phi_i)
    
    return val

# MCMC algorithm 

In [75]:

def MCMC_AFEM(beta_true, number_of_iter, burn_in, sigma, num_points): 
  '''
  Builds a Markov Chain 

  Key Steps:
  1. Initialise a choice of Beta, beta_0 
  2. Compute likelihood of beta_0, using delta and beta_0_predicted
  3. Initialise the loop.
      - we propose a new beta_i from x* ~ Uniform(0.15, 0.85) and r ~ Uniform(0, 0.15)
      - compute y_i and g(beta_i)
      - compute likelihood using {y_i and g(beta_i)}
      - set alpha = min{1, likelihood }
  '''
  # set seed 
  np.random.seed(72)
  # range of uniform distribution 
  x_star_range = (0.3, 0.7) 
  r_range = (0.1, 0.2) 
  chain = []
  ddof_list = 0
  # compute delta 
  mesh_true , c_sol_true, ddof_true= sl.refinement_loop(0.00001, beta_true) 
  y_true = fe_solution_at_obs(c_sol_true, mesh_true, np.linspace(0.0, 1.0, num_points))
  delta = add_noise(y_true, num_points, sigma)

  # draw first copy of beta --> beta_0

  beta_0 = np.array([
          np.random.uniform(*x_star_range),
          np.random.uniform(*r_range)
      ])
  print("Beta_0:", beta_0)
  
  # initialise current observations and likelihood 
  mesh_0, c_sol_0, ddof_0 = sl.refinement_loop(0.001, beta = beta_0)
  y_0 = fe_solution_at_obs(c_sol_0, mesh_0, np.linspace(0.0, 1.0, num_points))
  phi_0 = phi(delta, y_0, sigma, num_points)
  print("phi_0:", phi_0)
  
  iter_count = 0
  acceptance_count = 0 
  acceptance_prob_history = []

  for i in range(number_of_iter):
    beta_proposal = np.array([
        np.random.uniform(*x_star_range),
        np.random.uniform(*r_range)
    ])
    print("beta proposal:", beta_proposal)
    mesh_proposal, c_sol_proposal, ddof_proposal = sl.refinement_loop(0.001, beta = beta_proposal)
    y_proposal = fe_solution_at_obs(c_sol_proposal, mesh_proposal, np.linspace(0.0, 1.0, num_points))
    phi_proposal = phi(delta, y_proposal, sigma, num_points)
    print("phi proposal:", phi_proposal)
    # compute acceptance probability 
    A = compute_A(phi_0, phi_proposal, sigma)
    acceptance_prob = min(1, A)
    acceptance_prob_history.append(acceptance_prob)
    print("acceptance probablity:", acceptance_prob)
    
    # Accept or reject the proposal
    if np.random.rand() < acceptance_prob:
      beta_0 = beta_proposal # update the current state as the last accepted proposal
      y_0 = y_proposal # update the current observations to the last accepted observation
      phi_0 = phi_proposal
      ddof_0 = ddof_proposal
      acceptance_count += 1
    
    # Record the current state.
    chain.append(beta_0.copy())
    print("Chain length:", len(chain))
    ddof_list += ddof_0
      
  avg_ddof = ddof_list/len(chain)
  chain = np.array(chain)
  # Compute the MCMC estimate as the mean of the samples after burn-in.
  beta_mcmc = np.mean(chain[burn_in:], axis=0)
  
  return chain, beta_mcmc, acceptance_count, avg_ddof


In [76]:
# number_of_iter = 10000
# burn_in = 2000
# sigma = 0.01
# num_points = 100
# beta_true = np.array([0.65, 0.15])

# chain, beta_mcmc, acceptance_count, avg_dof = MCMC_AFEM(beta_true, number_of_iter, burn_in, sigma, num_points)
# print("True beta:", beta_true )
# print("MCMC estimated beta:", beta_mcmc)
# print("Acceptance Count:", acceptance_count)
# print("ddof list:", avg_dof)

In [83]:
def MCMC(beta_true, number_of_iter, burn_in, sigma, num_points):
    '''
    Builds two MCMC chains (one using AFEM and one using a uniform mesh) using the same sequence of proposed β.
    
    Parameters:
      beta_true      : The true parameter used to generate the true data.
      number_of_iter : Total number of MCMC iterations.
      burn_in        : Number of iterations to discard as burn-in.
      sigma          : Noise standard deviation.
      num_points     : Number of observation points.
    
    Returns:
      chain_afem, beta_mcmc_afem, acceptance_prob_history_afem, acceptance_count_afem,
      chain_uniform, beta_mcmc_uniform, acceptance_prob_history_uniform, acceptance_count_uniform,
      ddof_list
    '''
    np.random.seed(72)
    
    # Define proposal ranges:
    x_star_range = (0.3, 0.7)
    r_range = (0.1, 0.2)
    
    # Initialize lists to store chain history and diagnostic info:
    chain_afem = []
    chain_uniform = []
    ddof_list = []
    
    # Compute true data (delta) for both forward models:
    mesh_true, c_sol_true, ddof_true = sl.refinement_loop(0.00001, beta_true)
    y_true = fe_solution_at_obs(c_sol_true, mesh_true, np.linspace(0.0, 1.0, num_points))
    delta = add_noise(y_true, num_points, sigma)
    
    uniform_mesh = np.linspace(0.0, 1.0, 33)  # 33 nodes
    # c_sol_uniform_true, fvect_true_uniform = sl.solve_scF_once(uniform_mesh, beta_true)
    # y_true_uniform = fe_solution_at_obs(c_sol_uniform_true, uniform_mesh, np.linspace(0.0, 1.0, num_points))
    # delta_uniform = add_noise(y_true_uniform, num_points, sigma)
    
    # Draw initial beta (common initial state for both chains)
    beta_0 = np.array([np.random.uniform(*x_star_range),
                       np.random.uniform(*r_range)])
    print("Beta_0:", beta_0)
    
    # Initialize current state for AFEM chain:
    beta_current_afem = beta_0.copy()
    mesh_0, c_sol_0, ddof_0 = sl.refinement_loop(0.001, beta=beta_0)
    y_current_afem = fe_solution_at_obs(c_sol_0, mesh_0, np.linspace(0.0, 1.0, num_points))
    phi_current_afem = phi(delta, y_current_afem, sigma, num_points)
    
    # Initialize current state for Uniform FEM chain:
    beta_current_uniform = beta_0.copy()
    c_sol_0_uniform, fvect_0_uniform = sl.solve_scF_once(uniform_mesh, beta_0)
    y_current_uniform = fe_solution_at_obs(c_sol_0_uniform, uniform_mesh, np.linspace(0.0, 1.0, num_points))
    phi_current_uniform = phi(delta, y_current_uniform, sigma, num_points)
    
    acceptance_count_afem = 0
    acceptance_count_uniform = 0
    acceptance_prob_history_afem = []
    acceptance_prob_history_uniform = []
    
    # MCMC loop:
    for i in range(number_of_iter):
        # Generate a common proposal for β:
        beta_proposal = np.array([np.random.uniform(*x_star_range),
                                  np.random.uniform(*r_range)])
        print("beta proposal:", beta_proposal)
        
        # Evaluate proposal with AFEM:
        mesh_proposal_afem, c_sol_proposal_afem, ddof_proposal_afem = sl.refinement_loop(0.001, beta=beta_proposal)
        y_proposal_afem = fe_solution_at_obs(c_sol_proposal_afem, mesh_proposal_afem, np.linspace(0.0, 1.0, num_points))
        ddof_list.append((beta_proposal, ddof_proposal_afem))
        phi_proposal_afem = phi(delta, y_proposal_afem, sigma, num_points)
        print("phi proposal (AFEM):", phi_proposal_afem)
        A_afem = compute_A(phi_current_afem, phi_proposal_afem, sigma)
        acceptance_prob_afem = min(1, A_afem)
        acceptance_prob_history_afem.append(acceptance_prob_afem)
        print("acceptance probability (AFEM):", acceptance_prob_afem)
        
        # Evaluate proposal with Uniform FEM:
        c_sol_proposal_uniform, fvect_proposal_uniform = sl.solve_scF_once(uniform_mesh, beta=beta_proposal)
        y_proposal_uniform = fe_solution_at_obs(c_sol_proposal_uniform, uniform_mesh, np.linspace(0.0, 1.0, num_points))
        phi_proposal_uniform = phi(delta, y_proposal_uniform, sigma, num_points)
        print("phi proposal (Uniform FEM):", phi_proposal_uniform)
        A_uniform = compute_A(phi_current_uniform, phi_proposal_uniform, sigma)
        acceptance_prob_uniform = min(1, A_uniform)
        acceptance_prob_history_uniform.append(acceptance_prob_uniform)
        print("acceptance probability (Uniform FEM):", acceptance_prob_uniform)
        
        # For AFEM chain, use an independent random draw:
        if np.random.rand() < acceptance_prob_afem:
            beta_current_afem = beta_proposal.copy()
            y_current_afem = y_proposal_afem.copy()
            phi_current_afem = phi_proposal_afem
            acceptance_count_afem += 1
        
        # For Uniform FEM chain, use another independent random draw:
        if np.random.rand() < acceptance_prob_uniform:
            beta_current_uniform = beta_proposal.copy()
            y_current_uniform = y_proposal_uniform.copy()
            phi_current_uniform = phi_proposal_uniform
            acceptance_count_uniform += 1
        
        # Record the current states in their respective chains:
        chain_afem.append(beta_current_afem.copy())
        chain_uniform.append(beta_current_uniform.copy())
        print("Chain lengths: AFEM =", len(chain_afem), ", Uniform =", len(chain_uniform))
    
    chain_afem = np.array(chain_afem)
    chain_uniform = np.array(chain_uniform)
    beta_mcmc_afem = np.mean(chain_afem[burn_in:], axis=0)
    beta_mcmc_uniform = np.mean(chain_uniform[burn_in:], axis=0)
    
    return (chain_afem, beta_mcmc_afem, acceptance_prob_history_afem, acceptance_count_afem,
            chain_uniform, beta_mcmc_uniform, acceptance_prob_history_uniform, acceptance_count_uniform,
            ddof_list)


In [84]:

number_of_iter = 2000
burn_in = 1000
sigma = 0.01
num_points = 100
beta_true = np.array([0.65, 0.15])

# Call the MCMC function (assuming it has been defined and imported as above)
(chain_afem, beta_mcmc_afem, acceptance_prob_history_afem, acceptance_count_afem,
 chain_uniform, beta_mcmc_uniform, acceptance_prob_history_uniform, acceptance_count_uniform,
 ddof_list) = MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)

# Print the results
print("True beta:", beta_true)
print("AFEM recovered beta (mean after burn-in):", beta_mcmc_afem)
print("Uniform FEM recovered beta (mean after burn-in):", beta_mcmc_uniform)
print("AFEM acceptance count:", acceptance_count_afem)
print("Uniform FEM acceptance count:", acceptance_count_uniform)
print("AFEM acceptance probability history (first 10 samples):", acceptance_prob_history_afem[:10])
print("Uniform FEM acceptance probability history (first 10 samples):", acceptance_prob_history_uniform[:10])


Beta_0: [0.42297268 0.15835503]
beta proposal: [0.54983213 0.13747841]
phi proposal (AFEM): 71.33818347125305
acceptance probability (AFEM): 0.03579605050949741
phi proposal (Uniform FEM): 73.20399199841253
acceptance probability (Uniform FEM): 0.006027504628489586
Chain lengths: AFEM = 1 , Uniform = 1
beta proposal: [0.53689484 0.13033403]
phi proposal (AFEM): 75.4733546771439
acceptance probability (AFEM): 0.0005727341454196756
phi proposal (Uniform FEM): 75.17033440775985
acceptance probability (Uniform FEM): 0.0008436569617937452
Chain lengths: AFEM = 2 , Uniform = 2
beta proposal: [0.58878484 0.17287218]
phi proposal (AFEM): 58.264883801762316
acceptance probability (AFEM): 1
phi proposal (Uniform FEM): 57.69442456907155
acceptance probability (Uniform FEM): 1
Chain lengths: AFEM = 3 , Uniform = 3
beta proposal: [0.43403773 0.1302069 ]
phi proposal (AFEM): 73.01430762857684
acceptance probability (AFEM): 3.930127327234683e-07
phi proposal (Uniform FEM): 73.34891320324132
acceptanc

In [85]:
import numpy as np
np.set_printoptions(threshold=np.inf)  # Show entire array

print(chain_afem[burn_in:, ])


[[0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.69484832 0.10227035]
 [0.68582633 0.11010589]
 [0.68582633 0.11010589]
 [0.68582633 0.11010589]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]
 [0.62378982 0.17850762]


In [ ]:
print()

# Heat Map of Beta potential 

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import plotly.graph_objects as go

# # Parameters for the parameter grid (x* and r)
# x_star_vals = np.linspace(0.3, 0.7, 20)   # More candidate x* values for finer resolution
# r_vals = np.linspace(0.1, 0.2, 5)          # More candidate r values

# # Create a mesh grid of parameter pairs
# X, R = np.meshgrid(x_star_vals, r_vals)

# # Global parameters (use the same parameters throughout)
# epsilon = 1e-3       # Refinement tolerance
# sigma = 0.01         # Noise standard deviation
# num_points = 100     # Number of observation points

# # Generate the "true" observation using beta_true
# beta_true = np.array([0.65, 0.15])
# mesh_true, c_sol_true, ddof = sl.refinement_loop(epsilon, beta_true)
# y_true = fe_solution_at_obs(c_sol_true, mesh_true, np.linspace(0.0,1.0, num_points))
# observations = y_true + add_noise(y_true, num_points, sigma)
# true_potential = phi(y_true, y_true, sigma, num_points)

# # Prepare an array to store the misfit potential values
# potential = np.zeros_like(X)

# # Loop over the grid of (x*, r) candidates
# for i in range(X.shape[0]):
#     for j in range(X.shape[1]):
#         beta_candidate = np.array([X[i, j], R[i, j]])
#         mesh_candidate, c_sol_candidate, ddof = sl.refinement_loop(epsilon, beta_candidate)
#         y_candidate = fe_solution_at_obs(c_sol_candidate, mesh_candidate, np.linspace(0.0, 1.0, num_points))
#         potential[i, j] = phi(y_true, y_candidate, sigma, num_points)

# # Create the 2D contour plot with Plotly
# fig = go.Figure(data=[
#     go.Contour(
#         x=x_star_vals,
#         y=r_vals,
#         z=potential,
#         colorscale='turbid',
#         contours=dict(showlabels=True),
#         colorbar=dict(title="Potential")
#     ),
#     go.Scatter(
#         x=[beta_true[0]],
#         y=[beta_true[1]],
#         mode='markers',
#         marker=dict(size=10, color='cyan'),
#         name='True beta'
#     )
# ])

# # Update layout with axis titles and overall title
# fig.update_layout(
#     title='2D Contour Plot of the Misfit Potential',
#     xaxis_title='x*',
#     yaxis_title='r',
#     width=1000,
#     height=1000
# )

# fig.show()


In [ ]:
mesh_true_test, c_sol_true_test, ddof_true_test = sl.refinement_loop(0.001, beta = np.array([0.65,0.15]))

In [ ]:
ddof_true_test

28

In [ ]:
import numpy as np
import multiprocessing as mp

def run_single_chain(seed, beta_true, number_of_iter, burn_in, sigma, num_points):
    # Print a start message for this chain.
    print(f"Starting chain with seed {seed}", flush=True)
    
    # Set the random seed to ensure independent chains.
    np.random.seed(seed)
    result = MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)
    
    # Print a finish message for this chain.
    print(f"Finished chain with seed {seed}", flush=True)
    return result  # expected to be (chain, beta_mcmc, acceptance_history, acceptance_count)

def run_multiple_chains(n_chains, beta_true, number_of_iter, burn_in, sigma, num_points):
    # Generate random seeds for each chain.
    seeds = np.random.randint(0, 10000, size=n_chains)
    # Create argument tuples for each chain.
    args = [
        (seed, beta_true, number_of_iter, burn_in, sigma, num_points)
        for seed in seeds
    ]
    
    # Limit the number of worker processes to the minimum of available cores and 4.
    n_workers = min(mp.cpu_count(), 4)
    print(f"Using {n_workers} worker processes to run {n_chains} chains.", flush=True)
    
    # Create a multiprocessing Pool and run chains in parallel.
    with mp.Pool(processes=n_workers) as pool:
        results = pool.starmap(run_single_chain, args)
    
    print("All chains completed.", flush=True)
    
    # Unpack the results. Each result should be a tuple like
    # (chain, beta_mcmc, acceptance_history, acceptance_count).
    chains, beta_mcmcs, acceptance_histories, acceptance_counts = zip(*results)
    return list(chains), list(beta_mcmcs), list(acceptance_histories), list(acceptance_counts)

if __name__ == '__main__':
    n_chains = 4  # For example, 8 chains will run in two rounds with 4 workers.
    number_of_iter = 10000
    burn_in = 5000
    sigma = 0.01 
    num_points = 100
    beta_true = np.array([0.65,0.15])
    
    chains, beta_mcmcs, acceptance_histories, acceptance_counts = run_multiple_chains(
        n_chains, beta_true, number_of_iter, burn_in, sigma, num_points
    )
    
    print("True beta:", beta_true)
    print("MCMC estimated beta:", beta_mcmcs)
    print("Acceptance Probability History:", acceptance_histories)
    print("Acceptance Count:", acceptance_counts)


Using 4 worker processes to run 4 chains.
